# Portfolio Backtesting - Interactive Analysis\nThis notebook demonstrates how to use the portfolio backtester for interactive analysis.

In [ ]:
# Import required libraries\nimport sys\nimport os\nsys.path.append('..')  # Add parent directory to path\n\nfrom portfolio_backtest import PortfolioBacktester\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nfrom datetime import datetime, timedelta\n%matplotlib inline

## 1. Define Your Portfolio

In [ ]:
# Define portfolio allocation\nportfolio = {\n    'SPY': {'weight': 30.0, 'type': 'ETF', 'name': 'S&P 500'},\n    'QQQ': {'weight': 20.0, 'type': 'ETF', 'name': 'Nasdaq-100'},\n    'NVDA': {'weight': 15.0, 'type': 'Stock', 'name': 'Nvidia'},\n    'MSFT': {'weight': 10.0, 'type': 'Stock', 'name': 'Microsoft'},\n    'AAPL': {'weight': 10.0, 'type': 'Stock', 'name': 'Apple'},\n    'GLD': {'weight': 10.0, 'type': 'Commodity', 'name': 'Gold'},\n    'BND': {'weight': 5.0, 'type': 'ETF', 'name': 'Bonds'}\n}\n\n# Backtest parameters\nSTART_DATE = '2020-01-01'\nEND_DATE = datetime.now().strftime('%Y-%m-%d')\nINITIAL_CAPITAL = 100000\n\nprint(f\"Portfolio defined with {len(portfolio)} assets\")\nprint(f\"Backtest period: {START_DATE} to {END_DATE}\")

## 2. Run Backtest

In [ ]:
# Create backtester instance\nbacktester = PortfolioBacktester(\n    portfolio_dict=portfolio,\n    start_date=START_DATE,\n    end_date=END_DATE,\n    initial_capital=INITIAL_CAPITAL,\n    verbose=True\n)\n\n# Run the backtest\nbacktester.run()

## 3. View Performance Metrics

In [ ]:
# Display metrics\nmetrics = backtester.calculate_metrics()\n\nprint(\"Portfolio Performance Metrics:\")\nprint(\"=\" * 40)\nfor metric, value in metrics.items():\n    print(f\"{metric:.<25} {value}\")\n\n# Create metrics dataframe for easier analysis\nmetrics_df = pd.DataFrame([metrics]).T\nmetrics_df.columns = ['Value']

## 4. Visualizations

In [ ]:
# Generate comprehensive visualizations\nbacktester.plot_performance()

## 5. Custom Analysis

In [ ]:
# Access portfolio value directly\nportfolio_value = backtester.portfolio_value\nportfolio_returns = backtester.portfolio_returns\n\n# Calculate rolling statistics\nrolling_mean = portfolio_returns.rolling(window=30).mean() * 252 * 100\nrolling_std = portfolio_returns.rolling(window=30).std() * np.sqrt(252) * 100\n\n# Plot rolling statistics\nfig, axes = plt.subplots(2, 1, figsize=(12, 8))\n\naxes[0].plot(rolling_mean.index, rolling_mean, label='30-Day Rolling Annual Return')\naxes[0].axhline(y=0, color='r', linestyle='--', alpha=0.3)\naxes[0].set_title('Rolling Annualized Return (30-day window)')\naxes[0].set_ylabel('Return (%)')\naxes[0].legend()\naxes[0].grid(True, alpha=0.3)\n\naxes[1].plot(rolling_std.index, rolling_std, label='30-Day Rolling Volatility', color='orange')\naxes[1].set_title('Rolling Volatility (30-day window)')\naxes[1].set_ylabel('Volatility (%)')\naxes[1].set_xlabel('Date')\naxes[1].legend()\naxes[1].grid(True, alpha=0.3)\n\nplt.tight_layout()\nplt.show()

## 6. Compare Different Allocations

In [ ]:
# Define multiple portfolio strategies\nstrategies = {\n    'Aggressive': {\n        'NVDA': {'weight': 30.0, 'type': 'Stock', 'name': 'Nvidia'},\n        'TSLA': {'weight': 25.0, 'type': 'Stock', 'name': 'Tesla'},\n        'QQQ': {'weight': 25.0, 'type': 'ETF', 'name': 'Nasdaq-100'},\n        'ARKK': {'weight': 20.0, 'type': 'ETF', 'name': 'ARK Innovation'}\n    },\n    'Conservative': {\n        'BND': {'weight': 40.0, 'type': 'ETF', 'name': 'Bonds'},\n        'SPY': {'weight': 30.0, 'type': 'ETF', 'name': 'S&P 500'},\n        'GLD': {'weight': 20.0, 'type': 'Commodity', 'name': 'Gold'},\n        'VIG': {'weight': 10.0, 'type': 'ETF', 'name': 'Dividend Growth'}\n    },\n    'Balanced': {\n        'SPY': {'weight': 40.0, 'type': 'ETF', 'name': 'S&P 500'},\n        'BND': {'weight': 30.0, 'type': 'ETF', 'name': 'Bonds'},\n        'VEA': {'weight': 20.0, 'type': 'ETF', 'name': 'International'},\n        'GLD': {'weight': 10.0, 'type': 'Commodity', 'name': 'Gold'}\n    }\n}\n\n# Run backtests for each strategy\nresults = {}\n\nfor strategy_name, strategy_portfolio in strategies.items():\n    print(f\"Running backtest for {strategy_name} strategy...\")\n    \n    bt = PortfolioBacktester(\n        portfolio_dict=strategy_portfolio,\n        start_date=START_DATE,\n        end_date=END_DATE,\n        initial_capital=INITIAL_CAPITAL,\n        verbose=False\n    )\n    \n    try:\n        bt.run()\n        metrics = bt.calculate_metrics()\n        results[strategy_name] = {\n            'portfolio_value': bt.portfolio_value,\n            'metrics': metrics\n        }\n    except Exception as e:\n        print(f\"  Error: {e}\")\n\n# Compare strategies\nif results:\n    plt.figure(figsize=(12, 6))\n    \n    for strategy_name, data in results.items():\n        portfolio_value = data['portfolio_value']\n        plt.plot(portfolio_value.index, portfolio_value, \n                label=f\"{strategy_name}\", linewidth=2)\n    \n    plt.title('Strategy Comparison')\n    plt.xlabel('Date')\n    plt.ylabel('Portfolio Value ($)')\n    plt.legend()\n    plt.grid(True, alpha=0.3)\n    plt.tight_layout()\n    plt.show()\n    \n    # Compare metrics\n    print(\"\\nStrategy Comparison:\")\n    print(\"=\" * 60)\n    for strategy_name, data in results.items():\n        print(f\"\\n{strategy_name}:\")\n        for metric in ['Total Return', 'Sharpe Ratio', 'Max Drawdown']:\n            if metric in data['metrics']:\n                print(f\"  {metric}: {data['metrics'][metric]}\")

## 7. Export Results

In [ ]:
# Export to CSV\nbacktester.export_results('notebook_results.csv')\nprint(\"Results exported to notebook_results.csv\")\n\n# Create summary DataFrame\nsummary_df = pd.DataFrame({\n    'Date': portfolio_value.index,\n    'Portfolio Value': portfolio_value.values,\n    'Daily Return (%)': portfolio_returns.values * 100\n})\n\n# Display first and last few rows\nprint(\"\\nFirst 5 days:\")\nprint(summary_df.head())\nprint(\"\\nLast 5 days:\")\nprint(summary_df.tail())\n\n# Calculate monthly summary\nmonthly_summary = summary_df.set_index('Date').resample('M').agg({\n    'Portfolio Value': 'last',\n    'Daily Return (%)': lambda x: (1 + x/100).prod() - 1\n})\nmonthly_summary['Daily Return (%)'] *= 100\nmonthly_summary.columns = ['End Value', 'Monthly Return (%)']\n\nprint(\"\\nMonthly Summary (last 12 months):\")\nprint(monthly_summary.tail(12))